In [1]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pypyodbc

Note: you may need to restart the kernel to use updated packages.


In [1]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.engine import URL
import pypyodbc
import pandas as pd

###################################

# declares the variables and defines them for the server connections, along with the table names that are going to be assigned
SERVER_NAME = 'tcp:tecafs.database.windows.net,1433'
DATABASE_NAME = 'TecafsSqlDatabase'
TABLE_NAME = 'clean_data'

###################################

# makes the connection to the database with the connection string; has the driver, server name, database name, id, and password
connection_string = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Uid={'tecafs2023'};
    Pwd={'Capstone50'};
"""
###################################

# attempted connection string, didn't work though

#connection_string1 = pypyodbc.connect("Driver={ODBC Driver 18 for SQL Server};Server=tcp:tecafs.database.windows.net,1433;Database=TecafsSqlDatabase;Uid=tecafs2023;Pwd={Capstone50};")

###################################


connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_string})
engine = create_engine(connection_url, module=pypyodbc)

In [2]:
import pandas as pd

metadata = MetaData()
metadata.reflect(bind=engine)

# Print the names of tables
print("Tables in the database:")
for table in metadata.tables.values():
    print(table.name)

Tables in the database:
BuildVersion
clean_data
ErrorLog
ftworthweather_data
ML_forecast
synthesize_forecast
User_Info
VDB_forecast
weather


In [3]:
from pathlib import Path
import pandas as pd

#Query the data from the clean_data table
query = "SELECT * FROM clean_data" 

#Execute the query and load data into a Dataframe
clean_data_df = pd.read_sql(query, engine)

#Print the Dataframe
print(clean_data_df)

# Write the modified Dataframe back to a new CSV file
data_folder = Path("/mnt/c/Users/wcans/milvus_compose/ERCOT_Hourly_Load_2022/")
output_csv_file_path = data_folder / 'vdb_sql_read_data_test.csv'
clean_data_df.to_csv(output_csv_file_path, index=False)

                    hour_end         coast         east     far_west  \
0    2013-01-01 01:00:00.003   7607.189871  1074.024285  1411.823159   
1    2013-01-01 01:59:59.997   7389.267519  1035.189180  1403.601093   
2    2013-01-01 03:00:00.000   7179.748411  1036.216739  1395.131213   
3    2013-01-01 04:00:00.003   7039.692524  1032.777439  1395.593012   
4    2013-01-01 04:59:59.997   6991.412764  1042.906443  1401.239529   
...                      ...           ...          ...          ...   
8755 2013-12-31 19:59:59.997  10825.268372  1461.888557  1874.226858   
8756 2013-12-31 21:00:00.000  10509.204046  1441.627026  1869.683907   
8757 2013-12-31 22:00:00.003  10146.767461  1424.347535  1862.118638   
8758 2013-12-31 22:59:59.997   9740.175233  1388.813409  1846.269656   
8759 2014-01-01 00:00:00.000   9412.826300  1366.733450  1830.719100   

           north       north_c     southern      south_c         west  \
0     763.069242  10370.322921  2206.945802  4368.528687   883

In [6]:
# Define the SQL query

query = """
    SELECT * FROM clean_data 
    WHERE DATEPART(month, hour_end) = 1
    AND DATEPART(year, hour_end) = 2013;
"""

# Execute the query and load data into a DataFrame
jan_data = pd.read_sql(query, engine)

# Print the DataFrame
print(jan_data)




query = """
    SELECT * FROM clean_data
    WHERE south_c >= 8000;
"""

# Execute the query and load data into a DataFrame
large_power = pd.read_sql(query, engine)

# Print the DataFrame
print(large_power)

                   hour_end        coast         east     far_west  \
0   2013-01-01 01:00:00.003  7607.189871  1074.024285  1411.823159   
1   2013-01-01 01:59:59.997  7389.267519  1035.189180  1403.601093   
2   2013-01-01 03:00:00.000  7179.748411  1036.216739  1395.131213   
3   2013-01-01 04:00:00.003  7039.692524  1032.777439  1395.593012   
4   2013-01-01 04:59:59.997  6991.412764  1042.906443  1401.239529   
..                      ...          ...          ...          ...   
738 2013-01-31 19:00:00.003  9548.444485  1335.978377  1507.014586   
739 2013-01-31 19:59:59.997  9657.368663  1379.143751  1552.869725   
740 2013-01-31 21:00:00.000  9523.275000  1382.992255  1565.362806   
741 2013-01-31 22:00:00.003  9155.456175  1326.649295  1548.945064   
742 2013-01-31 22:59:59.997  8622.066013  1244.068590  1503.230477   

          north       north_c     southern      south_c         west  \
0    763.069242  10370.322921  2206.945802  4368.528687   883.029500   
1    754.425986